In [93]:
#!/usr/bin/env python3

import math

import numpy as np
import torch
import torch.nn as nn 


from gpytorch.utils.broadcasting import _pad_with_singletons


class GaussHermiteQuadrature1D(nn.Module):
    """
    Implements Gauss-Hermite quadrature for integrating a function with respect to several 1D Gaussian distributions
    in batch mode. Within GPyTorch, this is useful primarily for computing expected log likelihoods for variational
    inference.

    This is implemented as a Module because Gauss-Hermite quadrature has a set of locations and weights that it
    should initialize one time, but that should obey parent calls to .cuda(), .double() etc.
    """

    def __init__(self, num_locs=156):
        super().__init__()
        self.num_locs = num_locs

        locations, weights = self._locs_and_weights(num_locs)

        self.locations = locations
        self.weights = weights

    def _apply(self, fn):
        self.locations = fn(self.locations)
        self.weights = fn(self.weights)
        return super(GaussHermiteQuadrature1D, self)._apply(fn)

    def _locs_and_weights(self, num_locs):
        """
        Get locations and weights for Gauss-Hermite quadrature. Note that this is **not** intended to be used
        externally, because it directly creates tensors with no knowledge of a device or dtype to cast to.

        Instead, create a GaussHermiteQuadrature1D object and get the locations and weights from buffers.
        """
        locations, weights = np.polynomial.legendre.leggauss(1000)
        locations = torch.Tensor(locations)
        weights = torch.Tensor(weights)
        return locations, weights

    def forward(self, func, a=0,b=1):
        """
        Runs Gauss-Hermite quadrature on the callable func, integrating against the Gaussian distributions specified
        by gaussian_dists.

        Args:
            - func (callable): Function to integrate
            - gaussian_dists (Distribution): Either a MultivariateNormal whose covariance is assumed to be diagonal
                or a :obj:`torch.distributions.Normal`.
        Returns:
            - Result of integrating func against each univariate Gaussian in gaussian_dists.
        """
        lower_bound = torch.Tensor([a])
        higher_bound =  torch.Tensor([b])

        locations = _pad_with_singletons(self.locations, num_singletons_before=0, num_singletons_after=lower_bound.dim())
        t = 0.5*(locations + 1)*(higher_bound - lower_bound) + lower_bound

        x1 = t
        fx = func(x1)
        weights = _pad_with_singletons(self.weights, num_singletons_before=0, num_singletons_after=fx.dim() - 1)
        #res = torch.dot(fx,self.weights) #
        """shifted_locs = 0.5 * (locations + 1)*(variances-means) + means
        log_probs = func(shifted_locs)
        weights = _pad_with_singletons(self.weights, num_singletons_before=0, num_singletons_after=log_probs.dim() - 1)
        """
        res = (fx * weights)
        res = res.sum(tuple(range(self.locations.dim())))

        return res * 0.5 * (higher_bound - lower_bound)
    


In [94]:
f=lambda x : x**2

from scipy.integrate import quad
x,w = np.polynomial.legendre.leggauss(156)
#x,w = np.polynomial.hermite.hermgauss(156)
t = 0.5*(x + 1)*(1 - 0) + 0
gauss = 0
x1 = np.array([t.T])
fx = np.asarray(f(x1))

print(np.dot(fx, w)*0.5)
for i in range(len(t)):
    gauss = gauss + w[i] * f(x[i])
print(gauss* 0.5*(1 - 0))
quad(f, 0, 1)


[0.33333333]
0.3333333333333368


(0.33333333333333337, 3.700743415417189e-15)

In [95]:
a=0
b=1
deg = 156
x, w = np.polynomial.legendre.leggauss(deg)
# Translate x values from the interval [-1, 1] to [a, b]
t = 0.5*(x + 1)*(b - a) + a
gauss = sum(w * f(t)) * 0.5*(b - a)
print(gauss)

0.3333333333333345


In [96]:
from torch.autograd import Variable
X=Variable(torch.rand((2)),requires_grad=True)
print(X)
g = lambda x : x**2 + X[0]**2 + X[1]**2
gg = GaussHermiteQuadrature1D()
res = gg.forward(g)

tensor([0.31657910346984863281, 0.47011446952819824219], requires_grad=True)


In [97]:
jacobian = Jacobian()
jacobian(X,res.unsqueeze(1))

tensor([[0.63315820693969726562, 0.94022893905639648438]],
       grad_fn=<CopySlices>)

In [98]:
from complex import *

In [99]:
def tanC(X):
    X = Complex(X,torch.zeros(X.size()))
    return ((X**4 - (X**2 * 16) + X * 5) * 0.5).re.sum(axis=-1)

In [100]:
def StyblinskiTangNN(X):
    return (0.5 * (torch.pow(X,4) - 16 * torch.pow(X,2) + 5 * X)).sum(axis=-1)

In [101]:
import random
from torch.autograd import Variable
X = Variable(8*torch.rand((2, 5)),requires_grad=True)
print(tanC(X))
print(StyblinskiTangNN(X))

tensor([ 603.66601562500000000000, 2181.36108398437500000000],
       grad_fn=<SumBackward1>)
tensor([ 603.66601562500000000000, 2181.36108398437500000000],
       grad_fn=<SumBackward1>)


In [102]:
def phi(u,kappa_,S0_,r_,tau_,theta_,rho_,sigma_,V0_):
    gamma_ = np.sqrt((sigma_**2)*(u**2+u*1j)+(kappa_-1j*rho_*sigma_*u)**2)
    a = 1j*u*np.log(S0_) + 1j*u*r_*tau_+kappa_*theta_*tau_*(kappa_-1j*rho_*sigma_*u)/(sigma_**2)
    b = (u**2+1j*u)*V0_/(gamma_*np.cosh(gamma_*tau_/2)/np.sinh(gamma_*tau_/2)+kappa_-1j*rho_*sigma_*u)
    c = np.cosh(gamma_*tau_/2)+((kappa_-1j*rho_*sigma_*u)/gamma_)*np.sinh(gamma_*tau_/2)
    d = 2*kappa_*theta_/(sigma_**2)
    return np.exp(a)*np.exp(-b)/(c**d)


In [103]:
phi(1,2,1,0.01,0.8,0.04,-0.7,0.1,0.04)

(0.9838261582136327-0.007584806771239272j)

In [104]:
def Csqrt(nb_complex):
    real_ = torch.sqrt(nb_complex.real)
    imag_ = torch.sqrt(nb_complex.imag)
    comp = Complex(real_,imag_)
    return comp

In [105]:
def TorchPhi(u,kappa_,S0_,r_,tau_,theta_,rho_,sigma_,V0_):
    size = kappa_.size()
    gamma_ = (Complex(u ** 2,u)*((torch.pow(sigma_,2)))+(Complex(kappa_,u*-1*rho_*sigma_)**2))**(1/2)
    a = Complex(torch.zeros(size,dtype=torch.float64),u*torch.log(S0_)) + Complex(torch.zeros(size,dtype=torch.float64),u*r_*tau_) + (Complex(kappa_,torch.zeros(kappa_.size(),dtype=torch.float64)) - Complex(torch.zeros(size,dtype=torch.float64),u*rho_*sigma_))*(kappa_*theta_*tau_)*(1/(sigma_**2))
    b = Complex((u**2)*V0_,u*V0_).div(gamma_ * TorchCosh((gamma_*tau_)*(1/2)).div(TorchSinh((gamma_*tau_)*(1/2))) + (Complex(kappa_,torch.zeros(kappa_.size()))- Complex(torch.zeros(size),rho_*sigma_*u))  )
    c = TorchCosh((gamma_*tau_)*(1/2)) + (Complex(kappa_,torch.zeros(kappa_.size()))- Complex(torch.zeros(size),rho_*sigma_*u)).div(gamma_)*TorchSinh((gamma_*tau_)*(1/2)) 
    d = 2*kappa_*theta_/(sigma_**2)
    return Torchexp(a) * Torchexp(-b)*(c**d)**(-1)

In [106]:
TorchPhi(torch.DoubleTensor([1]),torch.DoubleTensor([2]),torch.DoubleTensor([1]),torch.DoubleTensor([0.01]),
         torch.DoubleTensor([0.8]),torch.DoubleTensor([0.04]),torch.DoubleTensor([-0.7]),torch.DoubleTensor([0.1]),torch.DoubleTensor([0.04]))

Complex(tensor([0.98382615821363128550], dtype=torch.float64), tensor([-0.00758480677123868352], dtype=torch.float64))

In [107]:
def psi(nu_,alpha_,K_,kappa_,S0_,r_,tau_,theta_,rho_,sigma_,V0_):
    k_ = np.log(K_)
    F = phi(nu_-1j*(alpha_+1),kappa_,S0_,r_,tau_,theta_,rho_,sigma_,V0_)*np.exp(-1j*nu_*k_)
    d = (alpha_+1j*nu_)*(alpha_+1+1j*nu_)
    return np.exp(-r_*tau_-alpha_*k_)/np.pi*(F/d).real


In [108]:
psi(1,0.1,0.8,2,1,0.01,0.8,0.04,-0.7,0.1,0.04)

-0.08136154279218144

In [109]:
def TorchPsi(nu_,alpha_,K_,kappa_,S0_,r_,tau_,theta_,rho_,sigma_,V0_):
    k_ = torch.log(K_)
    F = TorchPhi(Complex(nu_,torch.ones(nu_.size())*-(alpha_+1)),kappa_,S0_,r_,tau_,theta_,rho_,sigma_,V0_) * Torchexp(Complex(torch.zeros(nu_.size()),-nu_*k_))
    d = Complex(alpha_,nu_) * Complex(alpha_+1,nu_)
    return ((torch.exp(-r_*tau_-alpha_*k_)/math.pi)*(F.div(d))).re

In [110]:
TorchPsi(torch.DoubleTensor([1,1]),0.1,torch.DoubleTensor([0.8,0.8]),torch.DoubleTensor([2,2]),torch.DoubleTensor([1,1]),torch.DoubleTensor([0.01,0.01]),
         torch.DoubleTensor([0.8,0.8]),torch.DoubleTensor([0.04,0.04]),torch.DoubleTensor([-0.7,-0.7]),torch.DoubleTensor([0.1,0.1]),torch.DoubleTensor([0.04,0.04]))

tensor([-0.08136154274221486793, -0.08136154274221486793], dtype=torch.float64)

In [111]:
def price_heston_fourier(alpha_,K_,kappa_,S0_,r_,tau_,theta_,rho_,sigma_,V0_,L_):
    x,w = np.polynomial.legendre.leggauss(1000)
    f = lambda nu_: psi(nu_,alpha_,K_,kappa_,S0_,r_,tau_,theta_,rho_,sigma_,V0_)
    t = 0.5*(x + 1)*(L_ - 0) + 0
    
    x1 = np.array([t.T])
    fx = np.asarray(f(x1))

    I = np.dot(fx, w)*0.5*(L_-0)
    J = quad(lambda nu_: psi(nu_,alpha_,K_,kappa_,S0_,r_,tau_,theta_,rho_,sigma_,V0_) , 0, L_)
    return I[0], J[0]

In [112]:
price_heston_fourier(0.1,85,1,100,0.02,0.25,0.1,-0.7,0.1,0.04,10)
#K_=i,alpha_=0.1,r_=0.01,tau_=.5,kappa_=1,S0_=100,theta_=0.1,rho_=-0.7,sigma_=0.1,V0_=0.04,L_=100

(15.361951052954073, 15.361951053089967)

In [113]:
"""from numpy import meshgrid, sqrt, diff
from numpy import inf, pi, exp, linspace, zeros, real, imag, array, log
from scipy.stats import norm
from scipy.integrate import quad
# from bsm import bsmprice


def heston_phi(k, tau, *parms):

	v, vbar, lambd, eta, rho = parms
	
	b = lambd + 1j*rho*eta*k
	d = sqrt( b**2 + (eta**2)*k*(k-1j) )
	g = (b - d)/(b + d)
	T_m = (b - d)/(eta**2)
	T = T_m * ( 1 - exp(-d*tau) )/( 1 - g*exp(-d*tau) )
	W = lambd * vbar * ( tau*T_m - 2*log( ( 1 - g*exp(-d*tau) )/( 1 - g ) )/(eta**2) )
	
	return exp(W + v*T)


def heston_phi_transform(tau, x, *parms):
	integrand = lambda k: 2 * real( exp(-1j*k*x) * heston_phi(k + 0.5*1j, tau, *parms) )/(k**2 + 1.0/4.0)
	return quad(integrand, 0, 1000)[0]


def heston_ucall(F, K, tau, *parms):
	'''Heston call'''
	x = log(F/K)
	return F - (sqrt(K*F)/(2*pi)) * heston_phi_transform(tau, x, *parms)


def heston_call(S, K, tau, r, q, *parms):
	'''Heston call'''
	F = S*exp((r-q)*tau)
	x = log(F/K)
	integral = heston_phi_transform(tau, x, *parms)
	return S * exp(-q*tau) - K * exp(-r*tau) * heston_ucall(F, K, tau, *parms)"""

"from numpy import meshgrid, sqrt, diff\nfrom numpy import inf, pi, exp, linspace, zeros, real, imag, array, log\nfrom scipy.stats import norm\nfrom scipy.integrate import quad\n# from bsm import bsmprice\n\n\ndef heston_phi(k, tau, *parms):\n\n\tv, vbar, lambd, eta, rho = parms\n\t\n\tb = lambd + 1j*rho*eta*k\n\td = sqrt( b**2 + (eta**2)*k*(k-1j) )\n\tg = (b - d)/(b + d)\n\tT_m = (b - d)/(eta**2)\n\tT = T_m * ( 1 - exp(-d*tau) )/( 1 - g*exp(-d*tau) )\n\tW = lambd * vbar * ( tau*T_m - 2*log( ( 1 - g*exp(-d*tau) )/( 1 - g ) )/(eta**2) )\n\t\n\treturn exp(W + v*T)\n\n\ndef heston_phi_transform(tau, x, *parms):\n\tintegrand = lambda k: 2 * real( exp(-1j*k*x) * heston_phi(k + 0.5*1j, tau, *parms) )/(k**2 + 1.0/4.0)\n\treturn quad(integrand, 0, 1000)[0]\n\n\ndef heston_ucall(F, K, tau, *parms):\n\t'''Heston call'''\n\tx = log(F/K)\n\treturn F - (sqrt(K*F)/(2*pi)) * heston_phi_transform(tau, x, *parms)\n\n\ndef heston_call(S, K, tau, r, q, *parms):\n\t'''Heston call'''\n\tF = S*exp((r-q)*tau

In [114]:
heston_phi(log(0.8),0.8,0.04,0.1,2,0.1,-0.7)

NameError: name 'heston_phi' is not defined

In [115]:
def TorchHestonFourier(alpha_,K_,kappa_,S0_,r_,tau_,theta_,rho_,sigma_,V0_,L_):
    f = lambda nu_: TorchPsi(nu_,alpha_,K_,kappa_,S0_,r_,tau_,theta_,rho_,sigma_,V0_)
    gg = GaussHermiteQuadrature1D()
    res = gg.forward(f,0,L_)
    return res

In [116]:
X = Variable(torch.DoubleTensor([
    [0.9,2,1,0.01,0.8,0.04,-0.7,0.1,0.04],
    [0.8,2,1,0.01,0.8,0.04,-0.7,0.1,0.04]
]),requires_grad=True)
res=TorchHestonFourier(0.1,X[:,0],X[:,1],X[:,2],X[:,3],X[:,4],X[:,5],X[:,6],X[:,7],X[:,8],1000)

In [117]:
from jacobian import Jacobian

In [118]:
#X = Variable(torch.DoubleTensor([0.8,0.8]),requires_grad=True)
torch.set_printoptions(20)
res = res.unsqueeze(1)
jacobian = Jacobian()
jacobian(X,res)

tensor([[[-7.12254643440246582031e-01, -2.34298364375717937946e-04,
           7.76405692100524902344e-01,  5.12823343276977539062e-01,
           4.07512150704860687256e-02,  3.33460956811904907227e-01,
          -1.87207444105297327042e-03,  1.10308416187763214111e-02,
           3.37495356798171997070e-01],
         [-8.73869001865386962891e-01, -3.10378207359462976456e-04,
           9.14401829242706298828e-01,  5.59276163578033447266e-01,
           2.56397332996129989624e-02,  1.72182023525238037109e-01,
          -2.08160118199884891510e-03,  1.51532730087637901306e-02,
           1.78429335355758666992e-01]]], grad_fn=<CopySlices>)

In [119]:
a = price_heston_fourier(0.1,0.9,2,1,0.01,0.8,0.04,-0.7,0.1-1e-5,0.04,1000)
b = price_heston_fourier(0.1,0.9,2,1,0.01,0.8,0.04,-0.7,0.1+1e-5,0.04,1000)
print('Derivatives w.r.t sigma {}'.format((b-a)/2e-5))

FloatingPointError: underflow encountered in true_divide

In [120]:
# Standard library imports
import os
import sys
import random

# Important directories
code_dir = os.path.dirname(os.getcwd())
deep_cal_dir = os.path.dirname(os.path.dirname(os.getcwd()))
# Allows to import my own module
sys.path.insert(0, code_dir)
from ann.helpers import open_data
import pandas as pd

K_min = 90
S_min = 100
tau_min = 80
rho_min = -95
rho_max = -0.05
kappa_min = 10
kappa_max = 200.0
nu_min = 1
nu_max = 40
gamma_min = 10
gamma_max = 40
v_min = 5
v_max = 50
K_min  = 70
K_max = 130
min_ = [rho_min,kappa_min,nu_min,gamma_min,v_min,K_min,S_min,tau_min,1]
max_ = [rho_max,kappa_max,nu_max,gamma_max,v_max,K_max,S_min,tau_min,1]
sample_heston = np.random.uniform(low=min_, high=max_, size=(2000,9))/100

value_ = {'rho':sample_heston[:,0],'kappa':sample_heston[:,1],'theta':sample_heston[:,2],  'sigma':sample_heston[:,3], 
          'V0':sample_heston[:,4],'K':sample_heston[:,5],'S':sample_heston[:,6],'T':sample_heston[:,7],'r':sample_heston[:,8]}
database = pd.DataFrame(value_)

database.to_csv('data/tmp_heston.csv',index=False)

data = open_data('data/tmp_heston.csv')
print(data.head())
data = data.to_numpy()
data = torch.Tensor(data)
train_data = Variable(data[:50],requires_grad=True)
train_data1 = data[:50].numpy()

        rho     kappa     theta     sigma        V0         K    S    T     r
0 -0.432033  0.107787  0.119479  0.200008  0.393657  1.258110  1.0  0.8  0.01
1 -0.119619  1.623339  0.183953  0.333236  0.374372  1.242856  1.0  0.8  0.01
2 -0.299656  1.175966  0.062628  0.248367  0.442251  0.988256  1.0  0.8  0.01
3 -0.162127  0.204167  0.237252  0.243942  0.110359  0.729402  1.0  0.8  0.01
4 -0.216987  0.214322  0.205209  0.292142  0.443907  0.917386  1.0  0.8  0.01


In [121]:
res=TorchHestonFourier(0.1,train_data[:,5],train_data[:,1],1.0,train_data[:,8],train_data[:,7],train_data[:,2],train_data[:,0],train_data[:,3],train_data[:,4],100)

TypeError: log(): argument 'input' (position 1) must be Tensor, not float

In [122]:
res1 = []
train_data1 = train_data.detach().numpy()
np.seterr(all='raise')
for i in train_data1:
    try :
        res1.append(price_heston_fourier(1,0.9,i[1],1,0.01,0.8,i[2],i[0],i[3],i[4],100))
    except:
        res1.append(np.nan)
res1

[(0.2628743824413125, 0.26287438244239675),
 (0.23772536397802277, 0.23772536397904606),
 (0.24320185011617376, 0.24320185011720483),
 (0.17705540265152325, 0.17705540265241945),
 (0.2728457074609856, 0.2728457074621129),
 (0.21512881855851157, 0.21512881855945934),
 (0.23654767195980653, 0.2365476719608081),
 (0.18739644516967974, 0.18739644517059065),
 (0.26076819579945615, 0.26076819580051463),
 (0.269011347630708, 0.26901134763179885),
 (0.27581983070560623, 0.2758198307067538),
 (0.2803176231755379, 0.2803176231767047),
 (0.22072521900785422, 0.22072521900883896),
 (0.24484636219616823, 0.24484636219719227),
 (0.22673549716689598, 0.22673549716789584),
 (0.18947063202880235, 0.18947063202971842),
 (0.24336890304513403, 0.24336890304616962),
 (0.24470991044362464, 0.2447099104446492),
 (0.2538825447369189, 0.2538825447379757),
 (0.2603806443124708, 0.2603806443135457),
 (0.22405599586489194, 0.22405599586585906),
 (0.25308834455542706, 0.25308834455646373),
 (0.20601744751719955, 0

In [121]:
i = train_data1[35]
price_heston_fourier(1,0.9,i[1],1,0.01,0.8,i[2],i[0],0.1,i[4],1)

0.1662261463203171

In [115]:
train_data1[35]

array([-0.38529328,  1.1330202 ,  0.3592636 ,  0.02043602,  0.27256924,
        1.2178296 ,  1.        ,  0.8       ,  0.01      ], dtype=float32)

In [74]:
torch.set_printoptions(20)
res = res.unsqueeze(1)
jac = Jacobian()
topop = []
jac(train_data,res)
for i in range(len(res)):
    if res[i] is np.nan:
        topop.append(i)
        #res1 = np.delete(res1,i,0)
        
res = np.delete(res,topop,0)

RuntimeError: Can't call numpy() on Variable that requires grad. Use var.detach().numpy() instead.

In [70]:
len(res1)

45

In [105]:
y = res[res[:, 0] > 0]


In [103]:
x = train_data[res[:, 0] > 0]


In [78]:
len(y)

42

In [95]:
topop = []
boolean = res[:, 0] > 0
for i in range(len(boolean)):
    if not boolean[i] :
        topop.append(i)

res1 = np.delete(res1,topop,0)

In [101]:
jac = Jacobian()
jac(x,y)

TypeError: can't assign a NoneType to a torch.FloatTensor